In [1]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
import torchvision
import torchvision.transforms as transforms
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import DistanceMetric
import os
import sys
sys.path.append('../models')
import ResNet as resnet
import DenseNet as densenet
import EfficientNet as efficientnet
import MobileNetV2 as mobilenet
import ViT as vit

In [2]:
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "2, 3"

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
##########################################
######### Configure Metric Setting ####### 
##########################################
batch_size = 64
weight_path = './weights/baseline/dp-sgd/cifar10_dpsgd_resnet18_100.pt'
model = torch.load(weight_path)
criterion = nn.CrossEntropyLoss()
member_path = '/media/data1/hyunjun/cifar-10/train/'
nonmember_path = '/media/data1/hyunjun/cifar-10/test/'
data = 'Cifar-10'
##########################################
##########################################

In [4]:
mean, std = (0.485, 0.456, 0.406), (0.229, 0.224, 0.225) 
if data == 'Cifar-10':
    mean, std = (0.491, 0.482, 0.447), (0.247, 0.243, 0.262)
elif data == 'CelebA':
    mean, std = (0.5, 0.5, 0.5), (0.5, 0.5, 0.5)

trans = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean, std),
])

model.to(device)
model.eval()

nonmember_set = torchvision.datasets.ImageFolder(root=nonmember_path, transform=trans)
nonmember_loader = DataLoader(nonmember_set, batch_size=batch_size, shuffle=True, drop_last=False,)
member_set = torchvision.datasets.ImageFolder(root=member_path, transform=trans)
member_loader = DataLoader(member_set, batch_size=batch_size, shuffle=True, drop_last=False,)

classes = member_set.classes

In [5]:
latent_vectors = [[] for _ in range(len(classes))]

model.eval()
with torch.no_grad():
    for data in member_loader:
        images, labels = data[0].to(device), data[1].to(device)
        features, _ = model(images)

        deep_features = features.cpu().numpy().tolist()
        actual = labels.cpu().numpy().tolist()
        
        for i in range(len(actual)):
            latent_vectors[actual[i]].append(list(deep_features[i]))
            
    for data in nonmember_loader:
        images, labels = data[0].to(device), data[1].to(device)
        features, _ = model(images)

        deep_features = features.cpu().numpy().tolist()
        actual = labels.cpu().numpy().tolist()
        
        for i in range(len(actual)):
            latent_vectors[actual[i]].append(list(deep_features[i]))

In [6]:
print("#####################################")
print("### Calculating Cosine Similarity ###")
print("#####################################")

total_macro_mean = 0.0
for i in range(len(latent_vectors)):
    cssm = cosine_similarity(np.array(latent_vectors[i]))
    total_macro_mean += np.mean(cssm)
    print(f"class: {classes[i]}, max: {np.max(cssm):.4f}, min: {np.min(cssm):.4f}, mean: {np.mean(cssm):.4f}, var: {np.var(cssm):.4f}")
print("Overall Macro Mean:", total_macro_mean / len(latent_vectors))

#####################################
### Calculating Cosine Similarity ###
#####################################
class: airplane, max: 1.0000, min: -0.9806, mean: 0.5407, var: 0.4529
class: automobile, max: 1.0000, min: -0.9752, mean: 0.2932, var: 0.4613
class: bird, max: 1.0000, min: -0.9775, mean: 0.2111, var: 0.5978
class: cat, max: 1.0000, min: -0.9750, mean: 0.2977, var: 0.4616
class: deer, max: 1.0000, min: -0.9787, mean: 0.3968, var: 0.5471
class: dog, max: 1.0000, min: -0.9694, mean: 0.3325, var: 0.4395
class: frog, max: 1.0000, min: -0.9814, mean: 0.6250, var: 0.3674
class: horse, max: 1.0000, min: -0.9781, mean: 0.3529, var: 0.3993
class: ship, max: 1.0000, min: -0.9784, mean: 0.7000, var: 0.2837
class: truck, max: 1.0000, min: -0.9723, mean: 0.3933, var: 0.4218
Overall Macro Mean: 0.41431476448131715


In [7]:
print("#####################################")
print("## Calculating Euclidean Distance ###")
print("#####################################")

dist = DistanceMetric.get_metric('euclidean')
total_macro_mean = 0.0
for i in range(len(latent_vectors)):
    euclidean = dist.pairwise(np.array(latent_vectors[i]))
    total_macro_mean += np.mean(euclidean)
    print(f"class: {classes[i]}, max: {np.max(euclidean):.4f}, min: {np.min(euclidean):.4f}, mean: {np.mean(euclidean):.4f}, var: {np.var(euclidean):.4f}")
print("Overall Macro Mean:", total_macro_mean / len(latent_vectors))

#####################################
## Calculating Euclidean Distance ###
#####################################
class: airplane, max: 4.0946, min: 0.0000, mean: 0.8626, var: 0.4405
class: automobile, max: 3.9442, min: 0.0000, mean: 0.9631, var: 0.4262
class: bird, max: 4.2645, min: 0.0000, mean: 1.0937, var: 0.5722
class: cat, max: 3.7517, min: 0.0000, mean: 0.9436, var: 0.3922
class: deer, max: 4.0068, min: 0.0000, mean: 0.9944, var: 0.5758
class: dog, max: 3.8359, min: 0.0000, mean: 0.8839, var: 0.3460
class: frog, max: 3.5458, min: 0.0000, mean: 0.7756, var: 0.3841
class: horse, max: 3.6265, min: 0.0000, mean: 0.8770, var: 0.3405
class: ship, max: 4.0105, min: 0.0000, mean: 0.7963, var: 0.3304
class: truck, max: 3.7830, min: 0.0000, mean: 0.8718, var: 0.3537
Overall Macro Mean: 0.9061817667417129


In [8]:
print("#####################################")
print("## Calculating Manhattan Distance ###")
print("#####################################")

dist = DistanceMetric.get_metric('manhattan')
total_macro_mean = 0.0
for i in range(len(latent_vectors)):
    manhattan = dist.pairwise(np.array(latent_vectors[i]))
    total_macro_mean += np.mean(manhattan)
    print(f"class: {classes[i]}, max: {np.max(manhattan):.4f}, min: {np.min(manhattan):.4f}, mean: {np.mean(manhattan):.4f}, var: {np.var(manhattan):.4f}")
print("Overall Macro Mean:", total_macro_mean / len(latent_vectors))

#####################################
## Calculating Manhattan Distance ###
#####################################
class: airplane, max: 11.1597, min: 0.0000, mean: 2.3083, var: 3.3234
class: automobile, max: 10.7279, min: 0.0000, mean: 2.6282, var: 3.2816
class: bird, max: 11.6259, min: 0.0000, mean: 2.9744, var: 4.3441
class: cat, max: 10.4245, min: 0.0000, mean: 2.5668, var: 3.0008
class: deer, max: 11.0689, min: 0.0000, mean: 2.7248, var: 4.4411
class: dog, max: 10.6098, min: 0.0000, mean: 2.4031, var: 2.6481
class: frog, max: 9.9937, min: 0.0000, mean: 2.1074, var: 2.9276
class: horse, max: 10.2193, min: 0.0000, mean: 2.4055, var: 2.6510
class: ship, max: 10.9669, min: 0.0000, mean: 2.1281, var: 2.4872
class: truck, max: 10.3912, min: 0.0000, mean: 2.3726, var: 2.7335
Overall Macro Mean: 2.461919687467027


In [9]:
print("#####################################")
print("## Calculating Chebyshev Distance ###")
print("#####################################")

dist = DistanceMetric.get_metric('chebyshev')
total_macro_mean = 0.0
for i in range(len(latent_vectors)):
    chebyshev = dist.pairwise(np.array(latent_vectors[i]))
    total_macro_mean += np.mean(chebyshev)
    print(f"class: {classes[i]}, max: {np.max(chebyshev):.4f}, min: {np.min(chebyshev):.4f}, mean: {np.mean(chebyshev):.4f}, var: {np.var(chebyshev):.4f}")
print("Overall Macro Mean:", total_macro_mean / len(latent_vectors))

#####################################
## Calculating Chebyshev Distance ###
#####################################
class: airplane, max: 2.1473, min: 0.0000, mean: 0.4850, var: 0.1240
class: automobile, max: 2.0965, min: 0.0000, mean: 0.5140, var: 0.1117
class: bird, max: 2.2381, min: 0.0000, mean: 0.5919, var: 0.1559
class: cat, max: 1.9342, min: 0.0000, mean: 0.5090, var: 0.1045
class: deer, max: 2.1173, min: 0.0000, mean: 0.5288, var: 0.1503
class: dog, max: 1.9817, min: 0.0000, mean: 0.4760, var: 0.0917
class: frog, max: 1.8409, min: 0.0000, mean: 0.4182, var: 0.1017
class: horse, max: 1.8475, min: 0.0000, mean: 0.4639, var: 0.0878
class: ship, max: 2.1809, min: 0.0000, mean: 0.4466, var: 0.0937
class: truck, max: 1.9647, min: 0.0000, mean: 0.4694, var: 0.0929
Overall Macro Mean: 0.4902857919222249
